# Architecture of the Webcam Motion Detector

Intro:- 
    This is a real world motion detector program in python which captures frames through the webcam and checks if there is some 
    activity happening , it also prints a graph which shows for how much time there was activity(motion) in the frames captured by the webcam of our machine.

Steps:-
1) Capture the first frame,convert it into a grayscale and store it in a variable(*this frame will be compared with all the future frames)
2) Capture the other frame,convert it into a grayscale and blur it using gaussian method for better accuracy.
3) Find the difference_frame(ie delta_frame) between the default frame(first_frame) and the current frame(Using an inbuilt method)
4) Display a new frame which shows the difference between two frames.
5) The pixels(objects) which are different in both frames will have a grey-white color . 
6) Now find all pixels in the difference_frame which has pixel value of >100.(*in grayscale 255-white ; 0-black)
7) Using these pixels create an outline drawing for all these pixels.
8) Iterate through all the outlines(countors) and check if the outline object has pixel size of >500.
9) If true: then draw a rectangle using the are of countors of objects having pixel size > 500.
10) Draw the rectangle in the orginal frame(the color frame).


In [2]:
import cv2

first_frame=None
video=cv2.VideoCapture(0)
#A video is an array of frames(image).
while True:
    check, frame = video.read()
    # gray -> The current frame
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #Blurring the image to increase accuracy in calculation of difference.
    gray =cv2.GaussianBlur(gray,(21,21),0) #Ignore the values these are used by default.
    
    #Store the first instance of the frame
    if(first_frame is None):
        first_frame = gray
        continue #To go to next frame(as we do not want other line of code to be executed,will work only once)
    
    
    #Difference between two frames(first and current)
    delta_frame=cv2.absdiff(first_frame,gray) #Difference frame.
    
    """"
    Assign a threshold:- . 
    #If the difference between each pixels of both frames is 30 then we will consider
    #it as a white pixel and rest black pixel to get more accuracy(for contours)
    """
    #Assign a threshold on delta_frame for pixels that have value = or > 30 assign value of 255(white color).
    thresh_frame = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1] 
    
    
    
    #Make the white pixels(objects) smoother and more accurate.
    #Iterations means how many times to go through the image to remove (!needed) objects.
    thresh_frame=cv2.dilate(thresh_frame,None, iterations =2) 
    
    #Finding all contours of disctinct object in the thresh_frame.
    #RETR_EXTERNAL :- Draw external contours on objects we will be finding.
    (cnts,_) = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
    #Iterating through each contour and filtering(removing) the contour which has an area of < 1000px (!real world objects)
    for contour in cnts:
        if cv2.contourArea(contour) < 1000: #You can chage the value if you want to detect large objects. 
            continue #Ignore the contour having area less than 1000px.
    
        #If contour has size >1000 find the dimensions(x,y,w,h) of that contour
        (x,y,w,h) = cv2.boundingRect(contour)
        
        #Draw a rectangle on the contour(of the original color frame) to identify the real world object.
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
    
  #  cv2.imshow("First Frame",first_frame)
  #  cv2.imshow("Current Frame",gray)    
  #  cv2.imshow("Difference Frame",delta_frame)
  #  cv2.imshow("Threshold Frame",thresh_frame)
    cv2.imshow("Color Frame",frame) #Objects are identified in this frame.
    
    key = cv2.waitKey(1) #Wait for 1 milliseconds for user input 
    if(key==ord('q')):
        break
video.release() #Stop recording
cv2.destroyAllWindows()

# Addition to Motion Detector Code 

In [3]:
import cv2,pandas
from datetime import datetime

first_frame=None
status_list=[None,None] #Declare status list to find in how may frames object was present. 
times_list = [] #To catch the time when object came in and went out.
df = pandas.DataFrame(columns=["Start","End"]) #Create a data structure.
video=cv2.VideoCapture(0)
#A video is an array of frames(image).
while True:
    check, frame = video.read()
    status = 0 #0 - When no object is present in the frame and Vice versa
    # gray -> The current frame
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #Blurring the image to increase accuracy in calculation of difference.
    gray =cv2.GaussianBlur(gray,(21,21),0) #Ignore the values these are used by default.
    
    #Store the first instance of the frame
    if(first_frame is None):
        first_frame = gray
        continue #To go to next frame(as we do not want other line of code to be executed,will work only once)
    
    
    #Difference between two frames(first and current)
    delta_frame=cv2.absdiff(first_frame,gray) #Difference frame.
    
    """"
    Assign a threshold:- . 
    #If the difference between each pixels of both frames is 30 then we will consider
    #it as a white pixel and rest black pixel to get more accuracy(for contours)
    """
    #Assign a threshold on delta_frame for pixels that have value = or > 30 assign value of 255(white color).
    thresh_frame = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1] 
    
    
    
    #Make the white pixels(objects) smoother and more accurate.
    #Iterations means how many times to go through the image to remove (!needed) objects.
    thresh_frame=cv2.dilate(thresh_frame,None, iterations =2) 
    
    #Finding all contours of disctinct object in the thresh_frame.
    #RETR_EXTERNAL :- Draw external contours on objects we will be finding.
    (cnts,_) = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
    #Iterating through each contour and filtering(removing) the contour which has an area of < 1000px (!real world objects)
    for contour in cnts:
        if cv2.contourArea(contour) < 10000: #You can chage the value if you want to detect large objects. 
            continue #Ignore the contour having area less than 10000px.
    
        status = 1 #Some Object detected in the frame
        #If contour has size >1000 find the dimensions(x,y,w,h) of that contour
        (x,y,w,h) = cv2.boundingRect(contour)
        
        #Draw a rectangle on the contour(of the original color frame) to identify the real world object.
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
        
    #Appending status to the status_list which will show in how frames an object was detected.    
    status_list.append(status)
    
    #For memory management, if we run webcam gor hours thousands of status will be recorded but we need only value of last two
    status_list = status_list[-2:]
    #If the object was [-1]==1 (not present) and [-2]==0 and (now is present)
    if status_list[-1]==1 and status_list[-2]==0:
        times_list.append(datetime.now())
        
    #If the object present and now is not present
    if status_list[-1]==0 and status_list[-2]==1:
        times_list.append(datetime.now())
    
    
  #  cv2.imshow("First Frame",first_frame)
  #  cv2.imshow("Current Frame",gray)    
  #  cv2.imshow("Difference Frame",delta_frame)
  #  cv2.imshow("Threshold Frame",thresh_frame)
    cv2.imshow("Color Frame",frame) #Objects are identified in this frame.
    
    key = cv2.waitKey(1) #Wait for 1 milliseconds for user input 
    if(key==ord('q')):
        if status==1: #Object is detected 
            times_list.append(datetime.now()) #As there will not be an end time for the last detected object currently active. 
        break
#Print a list which prints either 0 or 1 . 0-means no object detected , 1-means object was detected
print("Captured Frames 0-No object detected ;  1-Object Detected")
print(status_list)
print()
print("Start and End times of objects in the frames")
print(times_list) #Will print the start and end values.



#len() - returns length of the list ; range - iterate from which value to which value ; step - skip how many values here-(2)
for i in range(0,len(times_list),2):
    #Append Start time [i] and End time[i] to the panda dataframe in dictonary data sctructure.
    #ignore_index=True useful because we are skipping the end time index and directly jumping from 0 to 2.
    df = df.append({"Start":times_list[i],"End":times_list[i+1]},ignore_index=True)
    
df.to_csv("Times.csv") #Exporting our data frame to a csv file.
video.release() #Stop recording
cv2.destroyAllWindows()

Captured Frames 0-No object detected ;  1-Object Detected
[0, 0]

Start and End times of objects in the frames
[]


# Plotting the timings in a quad graph

In [4]:
from bokeh.plotting import figure, show , output_file
from bokeh.models import HoverTool,ColumnDataSource
import pandas
#df = pandas.read_csv("Times.csv")

#Converting data frame's start and end time to string in order to work with datetime type in HoverTool
df["Start_string"] = df["Start"].dt.strftime("%Y-%M-%D %H:%M:%S")
df["End_string"] = df["End"].dt.strftime("%Y-%M-%D %H:%M:%S")
#Tell bokeh that we are using df as our column data source.
cds = ColumnDataSource(df)

#print(df["Start"])
p = figure(x_axis_type='datetime',height=300,width=1100,title="Motion Graph")

p.yaxis.minor_tick_line_color=None #To remove ticks in the y axis
p.ygrid[0].ticker.desired_num_ticks=1 #To have only 1 grid in the y-axis

#Create a hover object
hover = HoverTool(tooltips=[("Start","@Start_string"),("End","@End_string")]) #@Start will fetch values from Data Frame's Start column.
p.add_tools(hover)
#Draw a quad(bar diagram) to plot start and end values.
q=p.quad(left="Start",right="End",bottom=0,top=1,color="green",source=cds)

output_file("Graph.html")
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [9]:
print("Shama Chawala")

Shama Chawala
